##

### Analyzing the Popularity and Effectiveness of Chess Openings
##### Wilson Kong

Chess is a game with near-infinite possibilities built solely upon an 8x8 tiled board and 16 game pieces to a side. 

In [100]:
## Imports necessary libraries for this project

import chess
import chess.pgn
import pandas as pd
import numpy as np
import ipywidgets as wgt
import matplotlib.pylab as plt
import re
from time import sleep
from sklearn import linear_model

We'll be obtaining our data from lichess'

In [ ]:
# Opens the PGN file where our data is stored.

pgn = open("jan2017.pgn")

In [103]:
game = chess.pgn.read_game(pgn)

board = game.board()

out = wgt.Output()

with out:
    display(board)

def update_board(button):
    global game

    with out:
        for move in game.mainline_moves():
            sleep(0.5)
            out.clear_output()
            game = game.next()
            display(game.board())

play = wgt.Button(
    value=0,
    min=0,
    max=len(list(game.mainline_moves())),
    step=1,
    interval=200,
    description="Play Game",
    disabled=False
)

button.on_click(update_board, remove=True)

display(wgt.VBox([out, button]))